In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
x_train = pd.read_csv("../input/digit-recognizer/train.csv")
x_test = pd.read_csv("../input/digit-recognizer/test.csv")

In [4]:
X = x_train.values

In [5]:
y = X[:, 0]
X = X[:, 1:]
X = X.reshape(42000, 28, 28, 1)

In [6]:
test = x_test.values
test = test[:, np.newaxis]
test = test.reshape(28000, 28, 28, 1)

In [7]:
#TODO: Image augmentation

In [8]:
cnn = tf.keras.models.Sequential()

#Convolutional layer
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu',
    input_shape=([28, 28, 1])))

#Max pooling
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,
    strides=2))

#Second convolutional layer
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,
    strides=2))

#Flattening layer

cnn.add(tf.keras.layers.Flatten())


#ANN/full connection

cnn.add(tf.keras.layers.Dense(
    units=128,
    activation='relu'))

#Output layer

cnn.add(tf.keras.layers.Dense(
    units=10,
    activation='softmax'))


2022-10-04 13:34:48.076565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 13:34:48.156799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 13:34:48.157581: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 13:34:48.159314: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
#compiling
cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

#training
cnn.fit(
    x=X,
    y = y,
    epochs=25)

2022-10-04 13:34:51.066699: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-10-04 13:34:52.203427: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1313/1313 [==============================] - 10s 3ms/step - loss: 0.4642 - accuracy: 0.9320
Epoch 2/25
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0757 - accuracy: 0.9777
Epoch 3/25
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0541 - accuracy: 0.9833
Epoch 4/25
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0458 - accuracy: 0.9860
Epoch 5/25
1313/1313 [==============================] - 3s 3ms/step - loss: 0.0406 - accuracy: 0.9876
Epoch 6/25
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0357 - accuracy: 0.9893
Epoch 7/25
1313/1313 [==============================] - 3s 3ms/step - loss: 0.0305 - accuracy: 0.9908
Epoch 8/25
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0275 - accuracy: 0.9920
Epoch 9/25
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0242 - accuracy: 0.9925
Epoch 10/25
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0252 - accura

In [10]:
predictions = cnn.predict(test)

In [11]:
pred_lab = np.argmax(predictions, axis=-1)
pred_lab.reshape(28000, 1)

array([[2],
       [0],
       [9],
       ...,
       [3],
       [9],
       [2]])

In [12]:
pred_df = pd.DataFrame(pred_lab, index=pd.RangeIndex(start=1, stop=len(pred_lab)+1))

In [13]:
pred_df.reset_index(inplace=True)
pred_df.rename(columns = {'index':'ImageId', 0:'Label'}, inplace = True)

In [14]:
pred_df.set_index('ImageId')

,Label
ImageId,
1,2
2,0
3,9
4,0
5,3
...,...
27996,9
27997,7
27998,3


In [15]:
pred_df.to_csv('Predictions.csv')